In [1]:
%pip install selenium
%pip install pandas
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from time import sleep
from urllib.parse import urljoin
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import json

/Users/nicolepi/Documents/Wing/find-the-expert/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def scrape_directory(url):
    driver = webdriver.Chrome()
    names = []
    bio_urls = []
    titles = []

    try:
        driver.get(url)

        last_first_name = None  # To track the first name on the page
        while True:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "dynamic-grid-listing-item"))
            )

            faculty_listings = driver.find_elements(By.CLASS_NAME, "dynamic-grid-listing-item")
            
            # Check if the page has content and get the first name
            if faculty_listings:
                first_name_on_page = faculty_listings[0].find_element(By.CSS_SELECTOR, "h2").text
                
                # If the first name hasn't changed, assume we are revisiting the same page
                if first_name_on_page == last_first_name:
                    print("Content hasn't changed. Ending scrape.")
                    break
                
                # Update the tracking variable
                #last_first_name = first_name_on_page

            for faculty in faculty_listings:
                try:
                    link_element = faculty.find_element(By.TAG_NAME, "a")
                    name = link_element.find_element(By.CSS_SELECTOR, "h2").text
                    title = link_element.find_element(By.XPATH, "./following-sibling::div").text
                    link = link_element.get_attribute("href")
                    bio_url = urljoin(url, link)

                    names.append(name)
                    titles.append(title)
                    bio_urls.append(bio_url)
                    print(f"Name: {name}, Title: {title}, URL: {bio_url}")
                except NoSuchElementException:
                    continue

            try:
                next_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Next"))
                )
                #driver.execute_script("arguments[0].scrollIntoView();", next_button)
                driver.execute_script("arguments[0].click();", next_button)
                next_button.click()
                # Wait until the first name of the first faculty listing changes, indicating new content is loaded
                WebDriverWait(driver, 10).until(
                    lambda d: d.find_elements(By.CLASS_NAME, "dynamic-grid-listing-item")[0].find_element(By.CSS_SELECTOR, "h2").text != last_first_name
                )
            except (TimeoutException, NoSuchElementException):
                print("Reached the end of the directory or 'Next' button is not clickable.")
                break

    except TimeoutException:
        print("Timeout while waiting for page to load")
    finally:
        driver.quit()

    df = pd.DataFrame({'Name': names, 'Title': titles, 'url': bio_urls})

    return df

def extract_biographies(df):
    driver = webdriver.Chrome()

    # Create a new column 'Research Summary'
    df['Research Summary'] = None  # Initialize with None

    for index, row in df.iterrows():
        bio_url = row['url']
        try:
            driver.get(bio_url)
            full_bio_text = ""

            # Find the original target section and extract text
            try:
                target_section = driver.find_element(By.CLASS_NAME, 'field--name-field-cu-wysiwyg')
                full_bio_text += target_section.text
            except NoSuchElementException:
                # If the target section is not found, continue without raising an error
                pass

            # Assign the 'Research Summary' to the corresponding row in the DataFrame
            if full_bio_text:
                df.at[index, 'Research Summary'] = full_bio_text
            else:
                df.at[index, 'Research Summary'] = None

        except TimeoutException:
            print(f"Timeout while loading {bio_url}")
        except Exception as e:
            print(f"Failed to extract biography from {bio_url}: {e}")

    driver.quit()
    return df

url = "https://anthropology.columbia.edu/content/faculty-directory"
df = scrape_directory(url)
df = extract_biographies(df)

Name: Nadia L. Abu El-Haj, Title: Professor, Barnard College, URL: https://anthropology.columbia.edu/content/nadia-l-abu-el-haj
Name: Lila Abu-Lughod, Title: Joseph L. Buttenwieser Professor of Social Science, URL: https://anthropology.columbia.edu/content/lila-abu-lughod
Name: Vanessa Agard-Jones, Title: Assistant Professor, URL: https://anthropology.columbia.edu/content/vanessa-agard-jones
Name: Naor H. Ben-Yehoyada, Title: Assistant Professor, URL: https://anthropology.columbia.edu/content/naor-h-ben-yehoyada
Name: Brian Boyd, Title: Senior Lecturer & Director of Museum Anthropology, URL: https://anthropology.columbia.edu/content/brian-boyd
Name: Partha Chatterjee, Title: Professor Emeritus of Anthropology and of Middle Eastern, South Asian and African Studies, URL: https://anthropology.columbia.edu/content/partha-chatterjee
Name: Hannah Chazin, Title: Assistant Professor, URL: https://anthropology.columbia.edu/content/hannah-chazin
Name: Myron Cohen, Title: Professor, URL: https://

ElementClickInterceptedException: Message: element click intercepted: Element <a href="" ng-click="selectPage(page + 1, $event)" ng-disabled="no...()||ngDisabled" uib-tabindex-toggle="" class="ng-binding" aria-label="Go to next page 3" disabled="disabled" tabindex="-1">Next</a> is not clickable at point (678, 590). Other element would receive the click: <div id="cu-privacy-notice-text">...</div>
  (Session info: chrome=123.0.6312.87)
Stacktrace:
0   chromedriver                        0x00000001033b441c chromedriver + 4326428
1   chromedriver                        0x00000001033ac8e4 chromedriver + 4294884
2   chromedriver                        0x0000000102fd8088 chromedriver + 278664
3   chromedriver                        0x0000000103020554 chromedriver + 574804
4   chromedriver                        0x000000010301ead8 chromedriver + 568024
5   chromedriver                        0x000000010301cc58 chromedriver + 560216
6   chromedriver                        0x000000010301c188 chr

In [ ]:
# Initialize the sentence embedder model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

# Assuming 'Research Summary' column contains the text data
df['Research Embedding'] = df['Research Summary'].apply(lambda x: model.encode([x])[0] if x else None)
gdf = df

In [ ]:
gdf = df
# Assuming 'gdf' is your original DataFrame

# Convert the 'Research Embedding' column to a list of lists
gdf['Research Embedding'] = gdf['Research Embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else None)

# Convert the DataFrame to a list of dictionaries
data_list = gdf.to_dict(orient='records')

#with open('faculty_data_slavic.json', 'w') as json_file:
#with open('faculty_data_astro.json', 'w') as json_file:
#with open('faculty_data_chem.json', 'w') as json_file:
#with open('faculty_data_eesc.json', 'w') as json_file:
#with open('faculty_data_physics.json', 'w') as json_file:
with open('faculty_data_afamstudies.json', 'w') as json_file:    
    json.dump(data_list, json_file)

In [ ]:
# Load the JSON file back into a DataFrame
#with open('faculty_data_slavic.json', 'r') as json_file:
#with open('faculty_data_astro.json', 'r') as json_file:
#with open('faculty_data_chem.json', 'r') as json_file:
#with open('faculty_data_eesc.json', 'r') as json_file:
#with open('faculty_data_physics.json', 'r') as json_file:
with open('faculty_data_afamstudies.json', 'r') as json_file:
    loaded_data_list = json.load(json_file)

df = pd.DataFrame(loaded_data_list)

# Now, gdf_loaded should have the 'Research Embedding' column in its original form
df.head()

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
proposal = ["""
medical image machine learning
            """]

proposal_embedding = model.encode(proposal)[0]

In [ ]:
# Calculate dot product for each row
df['Dot Product'] = df['Research Embedding'].apply(lambda x: np.dot(proposal_embedding, x) if x is not None else None)

# Sort DataFrame based on dot product scores in descending order
df_sorted = df.sort_values(by='Dot Product', ascending=False)

top_5 = df_sorted.head(5)

print(top_5.iloc[0]['Research Summary'])

top_5.head()

In [ ]:
residual = proposal_embedding - top_5.iloc[0]['Research Embedding']
# print(residual)
proposal_embedding = residual

In [ ]:
king = model.encode('king')
norm = np.linalg.norm(king)
king = king / norm

man = model.encode('man')
norm = np.linalg.norm(man)
man = man / norm

woman = model.encode('woman')
norm = np.linalg.norm(woman)
woman = woman / norm

queen = model.encode('queen')
norm = np.linalg.norm(queen)
queen = queen / norm

queen_test = king - man + woman
norm = np.linalg.norm(queen_test)
queen_test = queen_test / norm

print('\'king - man + woman\' match to \'king\': ', np.dot(king, queen_test))
print('\'king - man + woman\' match to \'queen\': ', np.dot(queen, queen_test))

In [ ]:
# Calculate dot product for each row
df['Dot Product'] = df['Research Embedding'].apply(lambda x: np.dot(proposal_embedding, x) if x is not None else None)

# Sort DataFrame based on dot product scores in descending order
df_sorted = df.sort_values(by='Dot Product', ascending=False)

top_5 = df_sorted.head(5)

print(top_5.iloc[0]['Research Summary'])

In [ ]:
df = df.drop(columns=['Dot Product'])
df.to_csv('faculty_data.csv', index=False)

In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def create_database():
    conn = sqlite3.connect('biographies.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS biographies (
            id INTEGER PRIMARY KEY,
            url TEXT UNIQUE,
            biography TEXT
        )
    ''')
    conn.commit()
    conn.close()

def insert_biography(url, biography):
    conn = sqlite3.connect('biographies.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO biographies (url, biography) VALUES (?, ?)", (url, biography))
        conn.commit()
    except sqlite3.IntegrityError:
        print(f"URL already exists in database: {url}")
    finally:
        conn.close()

def extract_biographies(bio_urls):
    driver = webdriver.Chrome()

    for bio_url in bio_urls:
        try:
            driver.get(bio_url)
            full_bio_text = ""

            try:
                body_section = driver.find_element(By.CLASS_NAME, 'body.col-lg-7')
                full_bio_text += body_section.text + " "
            except NoSuchElementException:
                pass

            try:
                target_section = driver.find_element(By.CLASS_NAME, 'col-lg-7.entity.entity-paragraphs-item.paragraphs-item-content')
                full_bio_text += target_section.text
            except NoSuchElementException:
                pass

            if full_bio_text:
                print(f"Storing biography from {bio_url}")
                insert_biography(bio_url, full_bio_text)

        except TimeoutException:
            print(f"Timeout while loading {bio_url}")
        except Exception as e:
            print(f"Failed to extract biography from {bio_url}: {e}")

    driver.quit()

# Create the database and table
create_database()

# Assuming you have a list of bio URLs
bio_urls = ["your_list_of_bio_urls_here"]
extract_biographies(bio_urls)